In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import parsl
import kale
import kale.workflow_objects
import kale.workflow_widgets
import kale.parsl_dflow

In [3]:
import networkx

In [4]:
workers = parsl.ThreadPoolExecutor()
kale_dfk = kale.parsl_dflow.KaleDFK(executors=[workers])

In [5]:
@parsl.App('python', kale_dfk)
def rand_add(*prev_list):
    """Add a random number to the previous ones."""
    import random
    import time
    
    # Random int between 0 & 10, inclusive.
    myrand = random.randint(0, 10)
    mysum = myrand + sum(prev_list)
    
    print("My number is {}. I was given {}. The sum is {}.\n".format(myrand, prev_list, mysum), end='')
    time.sleep(2)
    
    return mysum

In [6]:
kale_dfk.new_workflow("Random Tree")

In [7]:
rand_add(
    rand_add(
        rand_add(),
        rand_add()
    ),
    rand_add(
        rand_add(),
        rand_add()
    )
)

rand_add(
    rand_add(
        rand_add(),
        rand_add()
    ),
    rand_add(
        rand_add(),
        rand_add()
    )
)

Adding deps: <kale.workflow_objects.PythonFunctionTask object at 0x7fda021200b8> <- [<kale.workflow_objects.PythonFunctionTask object at 0x7fda021204e0>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120278>]
Adding deps: <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120e80> <- [<kale.workflow_objects.PythonFunctionTask object at 0x7fda02120b70>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120cf8>]
Adding deps: <kale.workflow_objects.PythonFunctionTask object at 0x7fda02130048> <- [<kale.workflow_objects.PythonFunctionTask object at 0x7fda021200b8>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120e80>]
Adding deps: <kale.workflow_objects.PythonFunctionTask object at 0x7fda021304e0> <- [<kale.workflow_objects.PythonFunctionTask object at 0x7fda021301d0>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02130358>]
Adding deps: <kale.workflow_objects.PythonFunctionTask object at 0x7fda02130908> <- [<kale.workflow_objects.

<AppFuture at 0x7fda02130a20 state=finished returned NoneType>

In [8]:
def parsl_app_after_futures(app, futures, dfk):
    """Execute fn after deps

    app is a parsl app which returns a future
    futures is a list of Parsl futures which must complete before execution
    dfk is the Parsl DataFlowKernel

    """

    @parsl.App('python', dfk)
    def wrapper(*depends):
        return app()
    
    wrapper.__name__ = app.__name__

    return wrapper(*futures)


def parsl_wrap(fn, dfk, *args, **kwargs):
    """Wrapper to generate Parsl dependencies.

    Args:
        fn: function to be wrapper
        depends: list of Parsl apps (or wrapped functions) which must execute first
        dfk: Parsl DataFlowKernel
        *args: args for fn
        **kwargs: kwargs for fn


    """

    @parsl.App('python', dfk)
    def wrapper(*depends):
        return fn(*args, **kwargs)

    wrapper.__name__ = fn.__name__
    
    return wrapper


In [9]:
def parsl_run(self, workflow):
    """Execute workflow via Parsl.
    So far, I'm assuming that we're only executing PythonFunctionTasks via Parsl.
    """

    workflow.futures = dict()

    # Topological sort guarantees that parent node
    # appears in list before child.
    # Therefore, parent futures will exist
    # before children futures.
    for task in networkx.dag.topological_sort(workflow.dag):
        # Reset futures before submission
        task.reset_future()

        #task.get_parsl_app()
        
        # Prepare function to be run by parsl
        wrapped_func = parsl_wrap(
            task.func,
            self.parsl_dfk,
            *task.args,
            **task.kwargs
        )

        # Determine dependencies
        depends = [
            workflow.futures[dep]
            for dep in task.dependencies[workflow]
        ]

        # Submit functions to Parsl & save futures
        workflow.futures[task] = parsl_app_after_futures(
            wrapped_func,
            depends,
            self.parsl_dfk,
        )


In [10]:
wf = kale_dfk.kale_workflow

wp = kale.workflow_objects.WorkerPool(
    name='WorkerPool',
    wf_executor='parsl',
    num_workers=4
)

In [11]:
parsl_run(wp, wf)

My number is 7. I was given (). The sum is 7.
My number is 8. I was given (). The sum is 8.
My number is 6. I was given (). The sum is 6.


In [12]:
wpw = kale.workflow_widgets.WorkerPoolWidget()
wpw

WorkerPoolWidget(children=(HTML(value='<h3>Worker Pools</h3>'), TableWidget(children=(RowWidget(children=(HBox(children=(HTML(value='<b>Name</b>'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='<b>Location</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Executor</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Workers</b>'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(HTML(value='<b>Action</b>'),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(HTML(value='default'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='4'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='danger', description='Remove', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(Text(value='', layout=Layout(width='148px')),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('localhost',), value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('parsl', 'fireworks'), value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(IntText(value=1, layout=Layout(width='198px')),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='success', icon='plus', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px'))), layout=Layout(width='710px')), HTML(value='\n        <div class="alert alert-success" style="width: 710px">\n        WorkerPool \'default\' created.\n        </div>\n        ')))

In [13]:
ww = kale.workflow_widgets.WorkflowWidget(wf, wpw)
ww

WorkflowWidget(children=(VBox(children=(Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 2}, {'source': 1, 'target': 2}, {'source': 2, 'target': 6}, {'source': 3, 'target': 5}, {'source': 4, 'target': 5}, {'source': 5, 'target': 6}, {'source': 7, 'target': 9}, {'source': 8, 'target': 9}, {'source': 9, 'target': 13}, {'source': 10, 'target': 12}, {'source': 11, 'target': 12}, {'source': 12, 'target': 13}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '1', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '2', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fda021204e0>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120278>]', 'kwargs': '{}'}, {'label': '3', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '4', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '5', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fda02120b70>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120cf8>]', 'kwargs': '{}'}, {'label': '6', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fda021200b8>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02120e80>]', 'kwargs': '{}'}, {'label': '7', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '8', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '9', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fda021301d0>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02130358>]', 'kwargs': '{}'}, {'label': '10', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '11', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '12', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fda021305f8>, <kale.workflow_objects.PythonFunctionTask object at 0x7fda02130780>]', 'kwargs': '{}'}, {'label': '13', 'shape': 'rect', 'name': 'rand_add', 'task_

In [14]:
dag

My number is 9. I was given (). The sum is 9.
My number is 0. I was given (). The sum is 0.
My number is 5. I was given (7, 8). The sum is 20.
My number is 8. I was given (). The sum is 8.


NameError: name 'dag' is not defined

My number is 9. I was given (9, 6). The sum is 24.
My number is 4. I was given (). The sum is 4.
My number is 3. I was given (). The sum is 3.
My number is 10. I was given (8, 0). The sum is 18.
My number is 3. I was given (24, 20). The sum is 47.
My number is 8. I was given (3, 4). The sum is 15.
My number is 2. I was given (15, 18). The sum is 35.
